In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os, json
import contextlib
from langgraph.graph import StateGraph, END
from typing import Annotated, TypedDict
import operator
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AnyMessage, ToolMessage
from langchain_community.tools import TavilySearchResults
from langgraph.checkpoint.sqlite import SqliteSaver

In [3]:
tool = TavilySearchResults(tavily_api_key=os.getenv("TAVILY_API_KEY"),max_results=2)
memory = SqliteSaver.from_conn_string(":memory:")
checkpointer = contextlib.ExitStack().enter_context(memory)

In [4]:
from uuid import uuid4

In [5]:
def reduce_messages(left: list[AnyMessage], right: list[AnyMessage]) -> list[AnyMessage]:
    for message in right:
        if not message.id:
            message.id = str(uuid4())

    merged = left.copy()
    for message in right:
        for idx,existing in enumerate(merged):
            if existing.id == message.id:
                merged[idx] = message
                break
        else:
            merged.append(message)
    return merged

In [6]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage],reduce_messages]


In [7]:
print(tool)

max_results=2 api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********'))


In [8]:
class Agent:
    def __init__(self, model, tools, system="", checkpointer=None):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm",self.call_openai)
        graph.add_node("action",self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True:"action",False:END})
        graph.add_edge("action","llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer,interrupt_before=["action"])
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        print(f"State : {state}")
        result = state["messages"][-1]
        return len(result.tool_calls) > 0
    
    def call_openai(self, state: AgentState):
        messages = state["messages"]
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {"messages":[message]}
    
    def take_action(self, state: AgentState):
        tool_calls = state["messages"][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling {t}")
            result = self.tools[t["name"]].invoke(t["args"])
            results.append(ToolMessage(tool_call_id=t["id"],name=t['name'],content=str(result)))
        print("Back to model")
        return {"messages": results}

In [9]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatOpenAI(model="gpt-3.5-turbo",api_key=os.getenv("OPENAI_API_KEY"))
abot = Agent(model,[tool],system=prompt,checkpointer=checkpointer)

In [10]:
messages = [HumanMessage(content="What is the weather in narasaropet")]
thread = {"configurable":{"thread_id":"1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

State : {'messages': [HumanMessage(content='What is the weather in narasaropet', additional_kwargs={}, response_metadata={}, id='79236ced-ff5a-4226-814b-29eb0a6cd6d4'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_chOYtsfW1cWIDcUPWi0XE9Tc', 'function': {'arguments': '{"query":"weather in Narasaraopet"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 155, 'total_tokens': 180, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BLDKYdWiDamkzN6KJhKB4LiZ3Dy6Q', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f956db0b-e75e-4a0b-9d7a-5b0ca873b108-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'w

In [11]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='What is the weather in narasaropet', additional_kwargs={}, response_metadata={}, id='79236ced-ff5a-4226-814b-29eb0a6cd6d4'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_chOYtsfW1cWIDcUPWi0XE9Tc', 'function': {'arguments': '{"query":"weather in Narasaraopet"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 155, 'total_tokens': 180, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BLDKYdWiDamkzN6KJhKB4LiZ3Dy6Q', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f956db0b-e75e-4a0b-9d7a-5b0ca873b108-0', tool_calls=[{'name': 'tavily_search_results_json', 'args':

In [12]:
abot.graph.get_state(thread).next

('action',)

In [13]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

Calling {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Narasaraopet'}, 'id': 'call_chOYtsfW1cWIDcUPWi0XE9Tc', 'type': 'tool_call'}
Back to model
{'messages': [ToolMessage(content="[{'title': 'Narasaraopet weather in April 2025 ⋆ Narasaraopet temperature in ...', 'url': 'https://www.meteoprog.com/weather/Narasaraopet/month/april/', 'content': 'Narasaraopet (India) weather in April 2025 ☀️ Accurate weather forecast for Narasaraopet in ... Monthly weather forecast in Narasaraopet ... 11 Apr. +38°+27°. 12', 'score': 0.9304142}, {'title': 'Narasaraopet 14 Day Extended Forecast - Weather - Time and Date', 'url': 'https://www.timeanddate.com/weather/@1261848/ext', 'content': 'Apr 11, 106 / 80 °F, Showers early. Mostly cloudy. 108 °F, 8 mph, ↑, 26%, 21 ... * Updated Monday, April 7, 2025 8:54:04 pm Narasaraopet time - Weather by', 'score': 0.8891543}]", name='tavily_search_results_json', id='b2b9d42d-a74d-4713-bbdc-4b73c206af6d', tool_call_id='call_chOYtsfW1cWIDcUPWi0XE9T

In [14]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='What is the weather in narasaropet', additional_kwargs={}, response_metadata={}, id='79236ced-ff5a-4226-814b-29eb0a6cd6d4'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_chOYtsfW1cWIDcUPWi0XE9Tc', 'function': {'arguments': '{"query":"weather in Narasaraopet"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 155, 'total_tokens': 180, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BLDKYdWiDamkzN6KJhKB4LiZ3Dy6Q', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f956db0b-e75e-4a0b-9d7a-5b0ca873b108-0', tool_calls=[{'name': 'tavily_search_results_json', 'args':

In [15]:
abot.graph.get_state(thread).next

()

In [16]:
messages = [HumanMessage(content="How is weather in kondapur hyderabad")]
for event in abot.graph.stream({"messages":messages},thread):
    for v in event.values():
        print(v)
while abot.graph.get_state(thread).next:
    print(f"----{abot.graph.get_state(thread)}----")
    _input = input("proceed")
    if _input != "y":
        print("aborting")
        break
    for event in abot.graph.stream(None, thread):
        for v in event.values():
            print(v)

State : {'messages': [HumanMessage(content='What is the weather in narasaropet', additional_kwargs={}, response_metadata={}, id='79236ced-ff5a-4226-814b-29eb0a6cd6d4'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_chOYtsfW1cWIDcUPWi0XE9Tc', 'function': {'arguments': '{"query":"weather in Narasaraopet"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 155, 'total_tokens': 180, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BLDKYdWiDamkzN6KJhKB4LiZ3Dy6Q', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f956db0b-e75e-4a0b-9d7a-5b0ca873b108-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'w

# Modify State

In [18]:
messages = [HumanMessage(content="What is weather in guntur")]
thread = {"configurable":{"thread_id":"3"}}
for event in abot.graph.stream({"messages":messages},thread):
    for v in event.values():
        print(v)

State : {'messages': [HumanMessage(content='What is weather in guntur', additional_kwargs={}, response_metadata={}, id='e0c81b9f-d5f5-4a59-9629-f36f1405cc1f'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_pYA7sYXXFTZBGDQui2PErLLg', 'function': {'arguments': '{"query":"weather in Guntur"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 153, 'total_tokens': 176, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BLDciwRdt1zBgyt4Wu5RSDUeUIohx', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-de33aa74-66f6-4c77-815a-35b0dab15ed6-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Guntu

In [19]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='What is weather in guntur', additional_kwargs={}, response_metadata={}, id='e0c81b9f-d5f5-4a59-9629-f36f1405cc1f'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_pYA7sYXXFTZBGDQui2PErLLg', 'function': {'arguments': '{"query":"weather in Guntur"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 153, 'total_tokens': 176, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BLDciwRdt1zBgyt4Wu5RSDUeUIohx', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-de33aa74-66f6-4c77-815a-35b0dab15ed6-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'wea

In [20]:
current_values = abot.graph.get_state(thread)

In [23]:
current_values.values["messages"][-1]

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_pYA7sYXXFTZBGDQui2PErLLg', 'function': {'arguments': '{"query":"weather in Guntur"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 153, 'total_tokens': 176, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BLDciwRdt1zBgyt4Wu5RSDUeUIohx', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-de33aa74-66f6-4c77-815a-35b0dab15ed6-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Guntur'}, 'id': 'call_pYA7sYXXFTZBGDQui2PErLLg', 'type': 'tool_call'}], usage_metadata={'input_tokens': 153, 'output_tokens': 23, 'total_tokens': 176, 'input_token_d

In [24]:
current_values.values["messages"][-1].tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'weather in Guntur'},
  'id': 'call_pYA7sYXXFTZBGDQui2PErLLg',
  'type': 'tool_call'}]

In [26]:
_id = current_values.values["messages"][-1].tool_calls[0]["id"]

In [27]:
current_values.values["messages"][-1].tool_calls[0] = {'name': 'tavily_search_results_json','args': {'query': 'weather in Vijayawada'},'id': _id}

In [28]:
abot.graph.update_state(thread, current_values.values)

State : {'messages': [HumanMessage(content='What is weather in guntur', additional_kwargs={}, response_metadata={}, id='e0c81b9f-d5f5-4a59-9629-f36f1405cc1f'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_pYA7sYXXFTZBGDQui2PErLLg', 'function': {'arguments': '{"query":"weather in Guntur"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 153, 'total_tokens': 176, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BLDciwRdt1zBgyt4Wu5RSDUeUIohx', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-de33aa74-66f6-4c77-815a-35b0dab15ed6-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Vijay

{'configurable': {'thread_id': '3',
  'checkpoint_ns': '',
  'checkpoint_id': '1f017056-9840-695a-8002-3cc9a23b5754'}}

In [29]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='What is weather in guntur', additional_kwargs={}, response_metadata={}, id='e0c81b9f-d5f5-4a59-9629-f36f1405cc1f'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_pYA7sYXXFTZBGDQui2PErLLg', 'function': {'arguments': '{"query":"weather in Guntur"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 153, 'total_tokens': 176, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BLDciwRdt1zBgyt4Wu5RSDUeUIohx', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-de33aa74-66f6-4c77-815a-35b0dab15ed6-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'wea

In [30]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

Calling {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Vijayawada'}, 'id': 'call_pYA7sYXXFTZBGDQui2PErLLg', 'type': 'tool_call'}
Back to model
{'messages': [ToolMessage(content='[{\'title\': \'Weather in Vijayawada\', \'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Vijayawada\', \'region\': \'Andhra Pradesh\', \'country\': \'India\', \'lat\': 16.5167, \'lon\': 80.6167, \'tz_id\': \'Asia/Kolkata\', \'localtime_epoch\': 1744397390, \'localtime\': \'2025-04-12 00:19\'}, \'current\': {\'last_updated_epoch\': 1744397100, \'last_updated\': \'2025-04-12 00:15\', \'temp_c\': 29.1, \'temp_f\': 84.4, \'is_day\': 0, \'condition\': {\'text\': \'Thundery outbreaks in nearby\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/200.png\', \'code\': 1087}, \'wind_mph\': 8.1, \'wind_kph\': 13.0, \'wind_degree\': 139, \'wind_dir\': \'SE\', \'pressure_mb\': 1005.0, \'pressure_in\': 29.68, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 73

# Time travel

In [31]:
states = []
for state in abot.graph.get_state_history(thread):
    print(state)
    print("--------")
    states.append(state)

StateSnapshot(values={'messages': [HumanMessage(content='What is weather in guntur', additional_kwargs={}, response_metadata={}, id='e0c81b9f-d5f5-4a59-9629-f36f1405cc1f'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_pYA7sYXXFTZBGDQui2PErLLg', 'function': {'arguments': '{"query":"weather in Guntur"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 153, 'total_tokens': 176, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BLDciwRdt1zBgyt4Wu5RSDUeUIohx', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-de33aa74-66f6-4c77-815a-35b0dab15ed6-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'wea

In [32]:
to_reply = states[-3]

In [33]:
to_reply

StateSnapshot(values={'messages': [HumanMessage(content='What is weather in guntur', additional_kwargs={}, response_metadata={}, id='e0c81b9f-d5f5-4a59-9629-f36f1405cc1f'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_pYA7sYXXFTZBGDQui2PErLLg', 'function': {'arguments': '{"query":"weather in Guntur"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 153, 'total_tokens': 176, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BLDciwRdt1zBgyt4Wu5RSDUeUIohx', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-de33aa74-66f6-4c77-815a-35b0dab15ed6-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'wea

In [36]:
for event in abot.graph.stream(None, to_reply.config):
    for k,v in event.items():
        print(v)

Calling {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Guntur'}, 'id': 'call_pYA7sYXXFTZBGDQui2PErLLg', 'type': 'tool_call'}
Back to model
{'messages': [ToolMessage(content='[{\'title\': \'Weather in Guntur\', \'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Guntur\', \'region\': \'Andhra Pradesh\', \'country\': \'India\', \'lat\': 16.3, \'lon\': 80.45, \'tz_id\': \'Asia/Kolkata\', \'localtime_epoch\': 1744397781, \'localtime\': \'2025-04-12 00:26\'}, \'current\': {\'last_updated_epoch\': 1744397100, \'last_updated\': \'2025-04-12 00:15\', \'temp_c\': 29.2, \'temp_f\': 84.6, \'is_day\': 0, \'condition\': {\'text\': \'Partly Cloudy\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/116.png\', \'code\': 1003}, \'wind_mph\': 9.6, \'wind_kph\': 15.5, \'wind_degree\': 162, \'wind_dir\': \'SSE\', \'pressure_mb\': 1005.0, \'pressure_in\': 29.67, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 75, \'cloud\': 46, \'feelslike_c\

# Go back in time and edit

In [38]:
_id = to_reply.values["messages"][-1].tool_calls[0]["id"]

In [39]:
to_reply.values["messages"][-1].tool_calls[0] = {'name': 'tavily_search_results_json', 'args': {'query': 'weather in New york'}, 'id':_id}

In [40]:
branch_state = abot.graph.update_state(to_reply.config, to_reply.values)

State : {'messages': [HumanMessage(content='What is weather in guntur', additional_kwargs={}, response_metadata={}, id='e0c81b9f-d5f5-4a59-9629-f36f1405cc1f'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_pYA7sYXXFTZBGDQui2PErLLg', 'function': {'arguments': '{"query":"weather in Guntur"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 153, 'total_tokens': 176, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BLDciwRdt1zBgyt4Wu5RSDUeUIohx', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-de33aa74-66f6-4c77-815a-35b0dab15ed6-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in New y

In [41]:
for event in abot.graph.stream(None, branch_state):
    for k, v in event.items():
        if k != "__end__":
            print(v)

Calling {'name': 'tavily_search_results_json', 'args': {'query': 'weather in New york'}, 'id': 'call_pYA7sYXXFTZBGDQui2PErLLg', 'type': 'tool_call'}
Back to model
{'messages': [ToolMessage(content='[{\'title\': \'Weather in New York City in April 2025 - Detailed Forecast\', \'url\': \'https://www.easeweather.com/north-america/united-states/new-york/new-york-city/april\', \'content\': \'| Apr. 9 | \\nOvercast\\n| 44° /33° | 0\\xa0in | 2 |  |\\n| Apr. 10 | \\nSunny\\n| 48° /32° | 0\\xa0in | 2 |  |\\n| Apr. 11 | \\nSunny\\n| 51° /39° | 0\\xa0in | 3 |  |\\n| Apr. 12 | \\nPatchy rain possible\\n| 55° /44° | 0.01\\xa0in | 3 |  |\\n| Apr. 13 | \\nSunny\\n| 59° /44° | 0\\xa0in | 4 |  |\\n| Apr. 14 | \\nSunny\\n| 55° /37° | 0\\xa0in | 3 |  |\\n| Apr. 15 | \\nModerate or heavy rain shower\\n| 64° /50° | 0.2\\xa0in | 4 |  |\\n| Apr. 16 | \\nModerate rain at times\\n| 62° /48° | 0.08\\xa0in | 4 |  |\\n| Apr. 17 | \\nCloudy\\n| 59° /44° | 0.09\\xa0in | 4 |  |\\n| Apr. 18 | [...] April\\nJanuaryFebr

# Add Message to a state at a given time

In [42]:
to_reply

StateSnapshot(values={'messages': [HumanMessage(content='What is weather in guntur', additional_kwargs={}, response_metadata={}, id='e0c81b9f-d5f5-4a59-9629-f36f1405cc1f'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_pYA7sYXXFTZBGDQui2PErLLg', 'function': {'arguments': '{"query":"weather in Guntur"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 153, 'total_tokens': 176, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BLDciwRdt1zBgyt4Wu5RSDUeUIohx', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-de33aa74-66f6-4c77-815a-35b0dab15ed6-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'wea

In [43]:
_id = to_reply.values["messages"][-1].tool_calls[0]["id"]

In [47]:
state_update = {"messages": [ToolMessage(
    tool_call_id=_id,
    name="tavily_search_results_json",
    content="54 degree celcius",
)]}

In [49]:
branch_and_add = abot.graph.update_state(
    to_reply.config,
    state_update,
    as_node="action"
)

In [50]:
for event in abot.graph.stream(None, branch_and_add):
    for k,v in event.items():
        print(v)

State : {'messages': [HumanMessage(content='What is weather in guntur', additional_kwargs={}, response_metadata={}, id='e0c81b9f-d5f5-4a59-9629-f36f1405cc1f'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_pYA7sYXXFTZBGDQui2PErLLg', 'function': {'arguments': '{"query":"weather in Guntur"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 153, 'total_tokens': 176, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BLDciwRdt1zBgyt4Wu5RSDUeUIohx', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-de33aa74-66f6-4c77-815a-35b0dab15ed6-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Guntu